Football Injury Datasets

https://www.kaggle.com/datasets/smadler92/nfl-pfr

- Gives athletes, injuries, age, pos, games played, games started, weight, height
- Can create a model to predict injury likelihood, but not type of injuries

https://nflreadr.nflverse.com/articles/dictionary_injuries.html

- season type, pos, name (could create script to pull everything we want)

https://github.com/sammieerne/NFL-Injury-Analysis

- Good research